# Spark Data Wrangling

In [ ]:
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import * 

from env import get_db_url

## Acquire

In [ ]:
#create enviroment


### load mpg data set from pydataset

### write datafame to file

- `json`: for writing to a local json file(s)
- `csv`: for writing to a local csv file(s)
- `parquet`: Parquet is a very popular columnar storage format for Hadoop.
- `jdbc`: for writing to a SQL database table

#### write file to json

In [ ]:
#df.write.type


#### write dataframe to csv

In [ ]:
#df.write.format()


### read files
- spark.read.[type]

#### read json

#### read csv

### load source from 311_data in sql

In [ ]:
#sql query


In [ ]:
#make pandas df


In [ ]:
#create spark df


### data schema

In [ ]:
# from pyspark.sql.types import StructType, StructField, StringType

In [ ]:
#use structype to build a list of the structfield with the columns assocaited schema


### load cases from 311_data from sql

In [ ]:
#sql query


In [ ]:
#pandas df


In [ ]:
#spark df


## Prepare

- rename columns
- correct datatypes
- data transformation
- make new features
- join tables

### rename columns

#### change SLA_due_date to case_due_date

### correct datatypes

#### change close_closed and case_late columns into boolean values

In [1]:
#use condition to make true and false


#### change council_district datatype to string

In [2]:
#use .cast()


#### change dates to datetype

format date strings: https://docs.oracle.com/javase/10/docs/api/java/time/format/DateTimeFormatter.html

In [ ]:
#use to_timestamp


### data transformation

#### normalize address
- `lower`: lowercase everything
- `trim`: remove whitespace on the edges 

#### change num_days_late to num_weeks_late

#### change council_district to int and pad with 00s

In [ ]:
# '%03d' means at least 3 digits, pad with 0s


### new features

#### create zip code column

#### create case_lifetime column

- case_age: how long since the case first opened
- days_to_close: the number of days between days opened and days closed
- case_lifetime: if the case is open, how long since the case opened, if the case is closed, the number of days to close


In [ ]:
#use datediff() to find the difference between two dates


In [ ]:
#create days_to_close


In [ ]:
#look at new columns for cases that were closed


In [ ]:
#look at new columns for cases that were NOT closed


In [ ]:
#create case_lifetime column


In [ ]:
#drop unnecessary columns


### join the dept table from sql to our current df

In [ ]:
#get dept table from sql


In [ ]:
#join tables


### train, validate, test split

- `.randomSplit` to split df